In [1]:
import os
import re
import json
import math
from pathlib import Path
from collections import Counter, defaultdict
from typing import List, Dict, Tuple, Optional

import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
import joblib

from scipy import sparse

RAW_DIR   = Path("./Data")
OUT_DIR   = Path("./DataAfterProcessing")
PROC_DIR  = OUT_DIR / "Processed"

MODELS_DIR = Path("./Models")
CONTENT_DIR= MODELS_DIR / "Content"
RULES_DIR  = MODELS_DIR / "Rules"
for d in [CONTENT_DIR, RULES_DIR]:
    d.mkdir(parents=True, exist_ok=True)

products = pd.read_csv(PROC_DIR / "products_enriched.csv")
tx_long  = pd.read_csv(PROC_DIR / "transactions_long.csv")

print("products:", products.shape, "| tx_long:", tx_long.shape)

products: (12877, 15) | tx_long: (49786, 11)


In [2]:
import unicodedata

def strip_spaces(s: str) -> str:
    s = str(s).strip()
    s = re.sub(r"\s+", " ", s)
    return s

def remove_accents(s: str) -> str:
    s = str(s)
    s = unicodedata.normalize("NFD", s)
    s = "".join(ch for ch in s if unicodedata.category(ch) != "Mn")
    return unicodedata.normalize("NFC", s)

def normalize_text(s: str, lower=True, keep_accents=True) -> str:
    s = str(s)
    s = strip_spaces(s)
    if lower:
        s = s.lower()
    if not keep_accents:
        s = remove_accents(s)
    return s

# precompute normalized text for filtering (accent-insensitive)
products["full_text"] = products["full_metadata"].fillna("").astype(str)
products["full_text_norm"] = products["full_text"].apply(lambda x: normalize_text(x, keep_accents=False))
products["name_norm"] = products["product_name_vi"].fillna("").astype(str).apply(lambda x: normalize_text(x, keep_accents=False))

In [3]:
# =========================
# Vectorizers (nhẹ, nhanh, ổn cho tiếng Việt)
# =========================
WORD_VECT_PATH = CONTENT_DIR / "tfidf_word.joblib"
CHAR_VECT_PATH = CONTENT_DIR / "tfidf_char.joblib"
XW_PATH = CONTENT_DIR / "X_word.npz"
XC_PATH = CONTENT_DIR / "X_char.npz"
PROD_INDEX_PATH = CONTENT_DIR / "product_index.csv"

# Word TF-IDF: giữ dấu (để hiểu đúng "canh chua" vs "sua chua")
word_vec = TfidfVectorizer(
    lowercase=True,
    ngram_range=(1,2),
    min_df=2,
    max_df=0.95,
    token_pattern=r"(?u)\b\w+\b"
)

# Char TF-IDF: dùng bản bỏ dấu để tăng recall khi user gõ thiếu dấu
char_vec = TfidfVectorizer(
    lowercase=True,
    analyzer="char_wb",
    ngram_range=(3,5),
    min_df=2,
    max_df=0.95
)

texts_word = products["full_text"].fillna("").astype(str).tolist()
texts_char = products["full_text"].fillna("").astype(str).apply(lambda x: normalize_text(x, keep_accents=False)).tolist()

X_word = word_vec.fit_transform(texts_word).astype(np.float32)
X_char = char_vec.fit_transform(texts_char).astype(np.float32)

# chuẩn hóa cosine
X_word = normalize(X_word, norm="l2")
X_char = normalize(X_char, norm="l2")

joblib.dump(word_vec, WORD_VECT_PATH)
joblib.dump(char_vec, CHAR_VECT_PATH)
sparse.save_npz(XW_PATH, X_word)
sparse.save_npz(XC_PATH, X_char)

# mapping product_id -> row_idx để không bao giờ IndexError
prod_index = products[["product_id_str"]].copy()
prod_index["row_idx"] = np.arange(len(prod_index)).astype(int)
prod_index.to_csv(PROD_INDEX_PATH, index=False, encoding="utf-8")

print("Saved content artifacts to:", CONTENT_DIR.resolve())
print("X_word shape:", X_word.shape, "| X_char shape:", X_char.shape)

Saved content artifacts to: D:\TDTU\DA\DACNTT\Models\content
X_word shape: (12877, 128146) | X_char shape: (12877, 88843)


In [4]:
INTENT_KEYWORDS = {
    "cook":     ["nấu","làm","món","canh","xào","kho","chiên","lẩu","nướng","nguyên liệu","đi chợ"],
    "snack":    ["ăn vặt","snack","xem phim","kẹo","bánh","nước ngọt","trà sữa"],
    "skincare": ["da","mụn","sữa rửa mặt","kem chống nắng","tẩy trang","serum","dưỡng","toner"],
    "laundry":  ["giặt","nước giặt","bột giặt","nước xả","xả vải","viên giặt"],
    "cleaning": ["lau","dọn","tẩy","vệ sinh","lau sàn","rửa chén","nước tẩy"],
    "gift":     ["tặng","quà","biếu","sinh nhật","noel","valentine","tặng mẹ","tặng ba"],
    "mom_baby": ["bé","bỉm","tã","sữa bột","trẻ em","mẹ","em bé"],
    "pet":      ["chó","mèo","thú cưng","cát vệ sinh","pate","hạt","royal canin"]
}

# chặn nhầm "canh chua" -> "sữa chua"
PHRASE_GUARDS = [
    ("canh chua", ["sữa chua","yaourt","yogurt"]),
]

def detect_intent(q: str):
    q = q.lower()
    scores = {k:0 for k in INTENT_KEYWORDS}
    for intent, kws in INTENT_KEYWORDS.items():
        for kw in kws:
            if kw in q:
                scores[intent] += 1

    best_intent, best_score = max(scores.items(), key=lambda x:x[1])
    matched = [k for k,v in sorted(scores.items(), key=lambda x:-x[1]) if v>0]

    # guard: nếu canh chua có thì không ưu tiên skincare/dairy kiểu "sữa chua"
    for must_have, block_terms in PHRASE_GUARDS:
        if must_have in q:
            # không thay intent trực tiếp, nhưng giúp downstream không add category sai
            scores["skincare"] = 0

    return (best_intent if best_score>0 else "search"), matched, scores

def parse_budget_max(q: str) -> Optional[int]:
    q2 = q.lower().replace(".","").replace(",","")
    m = re.search(r"(dưới|<|<=)\s*(\d+)\s*(k|nghìn|ngan|tr|triệu)?", q2)
    if not m:
        return None
    val = int(m.group(2))
    unit = m.group(3)
    if unit in ["k","nghìn","ngan"]:
        return val*1000
    if unit in ["tr","triệu"]:
        return val*1000000
    return val

def parse_quantity_people(q: str) -> Optional[int]:
    m = re.search(r"(\d+)\s*(người|nguoi)", q.lower())
    return int(m.group(1)) if m else None

def extract_excludes(q: str) -> List[str]:
    ql = q.lower()
    ex = []
    # bắt: "đừng mua cá lóc", "không lấy bò", "trừ tôm"
    for m in re.findall(r"(đừng mua|không|trừ)\s+([^,;.]+)", ql):
        phrase = strip_spaces(m[1])
        phrase = re.sub(r"^mua\s+", "", phrase).strip()
        if phrase:
            ex.append(phrase)
    return ex[:8]

# include_terms gọn (không bơm category dài lê thê)
INTENT_SEED_TERMS = {
    "cook": ["nguyên liệu","gia vị","rau","thịt","cá","nước mắm"],
    "snack": ["snack","bánh","kẹo","nước ngọt"],
    "skincare": ["sữa rửa mặt","chống nắng","tẩy trang","serum"],
    "laundry": ["nước giặt","bột giặt","nước xả"],
    "cleaning": ["nước tẩy","rửa chén","lau sàn"],
    "gift": ["quà tặng","hộp quà","giỏ quà"],
    "mom_baby": ["bỉm","tã","sữa bột"],
    "pet": ["thức ăn chó","thức ăn mèo","cát vệ sinh"]
}

class QueryInterpreterLite:
    def analyze(self, query_raw: str) -> dict:
        q = strip_spaces(query_raw)
        intent, matched, scores = detect_intent(q)
        budget = parse_budget_max(q)
        people = parse_quantity_people(q)
        excludes = extract_excludes(q)

        include_terms = []
        include_terms.extend(INTENT_SEED_TERMS.get(intent, []))

        # thêm 1 ít keyword từ query (bỏ stopword cực cơ bản)
        stop = set(["muốn","cần","tìm","mua","cho","và","hoặc","một","ít","nhẹ","tối","nay","dưới","trên","giá","loại","đừng","không","trừ"])
        tokens = [t for t in re.findall(r"\w+", q.lower()) if t not in stop]
        include_terms.extend(tokens[:8])

        # unique + giới hạn
        seen = set()
        inc2 = []
        for t in include_terms:
            t = strip_spaces(t)
            if t and t not in seen:
                seen.add(t)
                inc2.append(t)
        inc2 = inc2[:15]

        return {
            "query_raw": query_raw,
            "query_norm": q.lower(),
            "intent": intent,
            "matched_intents": matched,
            "intent_scores": scores,
            "include_terms": inc2,
            "exclude_terms": excludes,
            "constraints": {
                "budget_max": budget,
                "quantity_people": people
            }
        }

qi = QueryInterpreterLite()
qi.analyze("muốn nấu canh chua cho 4 người, đừng mua cá lóc")

{'query_raw': 'muốn nấu canh chua cho 4 người, đừng mua cá lóc',
 'query_norm': 'muốn nấu canh chua cho 4 người, đừng mua cá lóc',
 'intent': 'cook',
 'matched_intents': ['cook'],
 'intent_scores': {'cook': 2,
  'snack': 0,
  'skincare': 0,
  'laundry': 0,
  'cleaning': 0,
  'gift': 0,
  'mom_baby': 0,
  'pet': 0},
 'include_terms': ['nguyên liệu',
  'gia vị',
  'rau',
  'thịt',
  'cá',
  'nước mắm',
  'nấu',
  'canh',
  'chua',
  '4',
  'người',
  'lóc'],
 'exclude_terms': ['cá lóc'],
 'constraints': {'budget_max': None, 'quantity_people': 4}}

In [9]:
# Load artifacts (nếu chạy notebook từ đầu thì đã có; vẫn load lại cho chắc)
word_vec = joblib.load(CONTENT_DIR / "tfidf_word.joblib")
char_vec = joblib.load(CONTENT_DIR / "tfidf_char.joblib")
X_word = sparse.load_npz(CONTENT_DIR / "X_word.npz")
X_char = sparse.load_npz(CONTENT_DIR / "X_char.npz")

prod_id_to_row = dict(zip(products["product_id_str"].astype(str), np.arange(len(products))))

def cosine_scores(query_text: str, w_word=0.55) -> np.ndarray:
    q_raw = strip_spaces(query_text)
    q_word = normalize(word_vec.transform([q_raw]).astype(np.float32), norm="l2")
    q_char = normalize(char_vec.transform([normalize_text(q_raw, keep_accents=False)]).astype(np.float32), norm="l2")

    sw = (X_word @ q_word.T).toarray().ravel()
    sc = (X_char @ q_char.T).toarray().ravel()
    return (w_word * sw + (1.0 - w_word) * sc)

def apply_excludes_mask(df: pd.DataFrame, excludes: List[str]) -> pd.Series:
    if not excludes:
        return pd.Series([True]*len(df), index=df.index)
    ex_norm = [normalize_text(x, keep_accents=False) for x in excludes if str(x).strip()]
    txt = df["full_text_norm"].fillna("").astype(str)
    ok = np.ones(len(df), dtype=bool)
    for ex in ex_norm:
        if ex:
            ok &= ~txt.str.contains(re.escape(ex), na=False)
    return pd.Series(ok, index=df.index)

def recommend_content(q_info: dict, k=10, return_scores=False) -> Tuple[pd.DataFrame, Optional[np.ndarray]]:
    # Aug query: query_norm + include_terms (gọn)
    base = q_info["query_norm"]
    inc = q_info.get("include_terms", [])
    aug = base + " " + " ".join(inc[:10])

    scores = cosine_scores(aug)
    df = products.copy()
    df["score_raw"] = scores

    # Boost nhẹ theo intent (KHÔNG FILTER CỨNG để tránh miss)
    intent = q_info.get("intent", "search")
    boost_parent = {
        "cook":     ["Thực Phẩm", "Rau", "Gia Vị", "Thịt", "Cá", "Hải Sản"],
        "snack":    ["Snack", "Ăn Vặt", "Bánh Kẹo", "Nước Ngọt"],
        "skincare": ["Chăm Sóc", "Làm Đẹp", "Chăm Sóc Da"],
        "laundry":  ["Giặt", "Xả Vải", "Bột Giặt", "Nước Giặt"],
        "cleaning": ["Vệ Sinh", "Tẩy Rửa", "Rửa Chén", "Lau Sàn"],
        "pet":      ["Thú Cưng", "Chó", "Mèo"],
        "mom_baby": ["Mẹ", "Bé", "Em Bé"],
        "gift":     ["Quà", "Giỏ Quà"]
    }.get(intent, [])

    parent_txt = df["parent_category_name"].fillna("") + " " + df["category_name"].fillna("") + " " + df["category_path"].fillna("")
    parent_norm = parent_txt.astype(str)
    intent_boost = np.zeros(len(df), dtype=np.float32)
    for key in boost_parent:
        intent_boost += parent_norm.str.contains(key, case=False, na=False).astype(np.float32).values
    df["score"] = df["score_raw"] + 0.03 * intent_boost

    # Budget
    bmax = q_info.get("constraints", {}).get("budget_max", None)
    if isinstance(bmax, (int,float)) and bmax and bmax > 0:
        df = df[df["price"].fillna(0).astype(int) <= int(bmax)].copy()

    # Excludes
    mask = apply_excludes_mask(df, q_info.get("exclude_terms", []))
    df = df[mask].copy()

    # Popularity rerank (nhẹ, tránh long-tail bị chìm hoàn toàn)
    df["pop"] = df.get("popularity_norm", 0).fillna(0).astype(float)
    df["final_score"] = df["score"] + 0.02 * df["pop"]

    out = df.sort_values("final_score", ascending=False).head(k).copy()
    out = out[[
        "product_id_str","product_name_vi","category_name","parent_category_name",
        "brand_name","price","final_score"
    ]].rename(columns={"final_score":"score"})
    if return_scores:
        return out, scores
    return out, None

# test nhanh
q_info = qi.analyze("Muốn tăng quà sinh nhật cho mẹ, dưới 500k")
rec_df, scores_vec = recommend_content(q_info, k=8, return_scores=True)
rec_df

,product_id_str,product_name_vi,category_name,parent_category_name,brand_name,price,score
2719,683123af419bc51ab2ee6716,Hộp Quà Rạng Rỡ Choice L,Chăm Sóc Khác,"Sức Khỏe, Làm Đẹp",Choice L,249000,0.176736
2723,683123b1419bc51ab2ee671c,Hộp Quà Yêu Kiều Choice L,Chăm Sóc Khác,"Sức Khỏe, Làm Đẹp",Choice L,189900,0.160657
2496,68312321419bc51ab2ee661e,Băng Vệ Sinh Diana Mama Cho Mẹ Sau Sinh Không ...,"Băng Vệ Sinh, Tampon, Tã Người Lớn",Vệ Sinh Phụ Nữ,Diana,36000,0.125698
12259,68313d52419bc51ab2ee902d,Túi Đựng Quà Tết 42 Greenwood 40 x 32 x 13cm,Dụng Cụ Văn Phòng Khác,Dụng Cụ Văn Phòng,Greenwood,27900,0.108849
5904,68312c02419bc51ab2ee74ec,Giỏ Xách Nachi Có Nắp Quai Giữa (Giao Màu Ngẫu...,Đồ Nội Thất Khác,Nội Thất,Inochi,111900,0.108392
510,68311e02419bc51ab2ee5d0d,Bánh Đậu Xanh Rồng Vàng Hoàng Gia 300G,Bánh Quy,Bánh Các Loại,Hoàng Gia,42700,0.106403
12262,68313d52419bc51ab2ee9030,Túi Đựng Quà Tết 43 Greenwood 40 x 32 x 16cm,Dụng Cụ Văn Phòng Khác,Dụng Cụ Văn Phòng,Greenwood,31900,0.105830
11378,68313aeb419bc51ab2ee8c75,Thú Nhồi Bông 202 (Giao Mẫu Ngẫu Nhiên),Thú Bông,"Đồ Chơi, Thú Bông",UNKNOWN_BRAND,192000,0.105568


In [ ]:
MIN_COCOUNT_ITEM = 5
MIN_CONF_ITEM = 0.08
MIN_LIFT_ITEM = 1.05

MIN_COCOUNT_CAT = 8
MIN_CONF_CAT = 0.10
MIN_LIFT_CAT = 1.05

def build_bill_items(tx_long: pd.DataFrame) -> Dict[int, List[str]]:
    g = tx_long.groupby("bill_id")["product_id_str"].apply(lambda s: sorted(set(s.astype(str))))
    return g.to_dict()

bill_items = build_bill_items(tx_long)
n_bills = len(bill_items)
print("n_bills:", n_bills)

def mine_item_rules_paircount(bill_items: Dict[int, List[str]]) -> pd.DataFrame:
    item_count = Counter()
    pair_count = Counter()

    for _, items in bill_items.items():
        items = [x for x in items if x]
        items = sorted(set(items))
        for a in items:
            item_count[a] += 1
        # unordered pairs
        m = len(items)
        for i in range(m):
            for j in range(i+1, m):
                pair_count[(items[i], items[j])] += 1

    rows = []
    for (a,b), co in pair_count.items():
        # a->b
        for x,y in [(a,b),(b,a)]:
            co_xy = co
            if co_xy < MIN_COCOUNT_ITEM:
                continue
            conf = co_xy / item_count[x]
            if conf < MIN_CONF_ITEM:
                continue
            supp = co_xy / n_bills
            lift = conf / (item_count[y] / n_bills)
            if lift < MIN_LIFT_ITEM:
                continue
            leverage = supp - (item_count[x]/n_bills)*(item_count[y]/n_bills)
            rows.append([x,y,co_xy,item_count[x],item_count[y],supp,conf,lift,leverage])

    df = pd.DataFrame(rows, columns=[
        "antecedent_id","consequent_id","co_count","ante_count","cons_count",
        "support","confidence","lift","leverage"
    ])

    # map names
    name_map = products.set_index("product_id_str")["product_name_vi"].astype(str).to_dict()
    df["antecedent_name"] = df["antecedent_id"].map(name_map)
    df["consequent_name"] = df["consequent_id"].map(name_map)

    # sort best rules
    df = df.sort_values(["lift","confidence","support"], ascending=False).reset_index(drop=True)
    return df

item_rules = mine_item_rules_paircount(bill_items)
print("item_rules:", item_rules.shape)
display(item_rules.head(20))

item_rules.to_csv(RULES_DIR / "item_rules.csv", index=False, encoding="utf-8")
print("Saved:", RULES_DIR / "item_rules.csv")

n_bills: 4430
item_rules: (258, 11)


,antecedent_id,consequent_id,co_count,ante_count,cons_count,support,confidence,lift,leverage,antecedent_name,consequent_name
0,68312cde419bc51ab2ee7649,68312802419bc51ab2ee6e77,5,12,24,0.001129,0.416667,76.909722,0.001114,Bột Giặt Surf Hương Nước Hoa Quyến Rũ Túi 5.3kg,Nước Xả Vải Lix Sạch Thơm Ngàn Hoa Túi 2.2L
1,68312802419bc51ab2ee6e77,68312cde419bc51ab2ee7649,5,24,12,0.001129,0.208333,76.909722,0.001114,Nước Xả Vải Lix Sạch Thơm Ngàn Hoa Túi 2.2L,Bột Giặt Surf Hương Nước Hoa Quyến Rũ Túi 5.3kg
2,6831396e419bc51ab2ee8a09,6831230f419bc51ab2ee65f9,7,18,44,0.001580,0.388889,39.154040,0.001540,Khăn Giấy Bếp Đa Năng Fairy 2 Lớp 100 Tờ,Giấy Vệ Sinh Gấu Trúc Silkwell Tre 3 Lớp 10 Cu...
3,6831230f419bc51ab2ee65f9,6831396e419bc51ab2ee8a09,7,44,18,0.001580,0.159091,39.154040,0.001540,Giấy Vệ Sinh Gấu Trúc Silkwell Tre 3 Lớp 10 Cu...,Khăn Giấy Bếp Đa Năng Fairy 2 Lớp 100 Tờ
4,68312772419bc51ab2ee6d81,68312309419bc51ab2ee65ef,5,13,45,0.001129,0.384615,37.863248,0.001099,Nước Giặt OMO Matic Cửa Trước Giữ Màu Túi 4.1kg,Giấy Vệ Sinh E'mos Classic 2 Lớp Lốc 10 Cuộn
5,68312309419bc51ab2ee65ef,68312772419bc51ab2ee6d81,5,45,13,0.001129,0.111111,37.863248,0.001099,Giấy Vệ Sinh E'mos Classic 2 Lớp Lốc 10 Cuộn,Nước Giặt OMO Matic Cửa Trước Giữ Màu Túi 4.1kg
6,6831277b419bc51ab2ee6d96,6831230a419bc51ab2ee65f1,5,15,40,0.001129,0.333333,36.916667,0.001098,Nước Giặt Ariel Chuyên Gia Cửa Trước Hương Dow...,Giấy Vệ Sinh Premier Deluxe 3 Lớp Lốc 10 Cuộn
7,6831230a419bc51ab2ee65f1,6831277b419bc51ab2ee6d96,5,40,15,0.001129,0.125000,36.916667,0.001098,Giấy Vệ Sinh Premier Deluxe 3 Lớp Lốc 10 Cuộn,Nước Giặt Ariel Chuyên Gia Cửa Trước Hương Dow...
8,6831230e419bc51ab2ee65f8,68312cd6419bc51ab2ee763f,5,48,16,0.001129,0.104167,28.841146,0.001090,Giấy Vệ Sinh Silkwell 4 Lớp Lốc 10 Cuộn,Bột Giặt Lix Sạch Thơm 24 Giờ Ngát Hương Túi 5...
9,68312cd6419bc51ab2ee763f,6831230e419bc51ab2ee65f8,5,16,48,0.001129,0.312500,28.841146,0.001090,Bột Giặt Lix Sạch Thơm 24 Giờ Ngát Hương Túi 5...,Giấy Vệ Sinh Silkwell 4 Lớp Lốc 10 Cuộn


Saved: Models\Rules\item_rules.csv


In [10]:
def build_bill_parentcats(tx_long: pd.DataFrame) -> Dict[int, List[str]]:
    g = tx_long.groupby("bill_id")["parent_category_name"].apply(
        lambda s: sorted(set([x for x in s.astype(str) if x and x != "nan"]))
    )
    return g.to_dict()

bill_cats = build_bill_parentcats(tx_long)
print("Example bill cats:", list(bill_cats.items())[:1])

def mine_cat_rules_paircount(bill_cats: Dict[int, List[str]]) -> pd.DataFrame:
    cat_count = Counter()
    pair_count = Counter()

    for _, cats in bill_cats.items():
        cats = sorted(set([c for c in cats if c]))
        for a in cats:
            cat_count[a] += 1
        m = len(cats)
        for i in range(m):
            for j in range(i+1, m):
                pair_count[(cats[i], cats[j])] += 1

    rows = []
    for (a,b), co in pair_count.items():
        for x,y in [(a,b),(b,a)]:
            if co < MIN_COCOUNT_CAT:
                continue
            conf = co / cat_count[x]
            if conf < MIN_CONF_CAT:
                continue
            supp = co / n_bills
            lift = conf / (cat_count[y] / n_bills)
            if lift < MIN_LIFT_CAT:
                continue
            leverage = supp - (cat_count[x]/n_bills)*(cat_count[y]/n_bills)
            rows.append([x,y,co,cat_count[x],cat_count[y],supp,conf,lift,leverage])

    df = pd.DataFrame(rows, columns=[
        "ante_parent_cat","cons_parent_cat","co_count","ante_count","cons_count",
        "support","confidence","lift","leverage"
    ])
    df = df.sort_values(["lift","confidence","support"], ascending=False).reset_index(drop=True)
    return df

cat_rules = mine_cat_rules_paircount(bill_cats)
print("cat_rules:", cat_rules.shape)
display(cat_rules.head(20))

cat_rules.to_csv(RULES_DIR / "category_rules.csv", index=False, encoding="utf-8")
print("Saved:", RULES_DIR / "category_rules.csv")


Example bill cats: [(0, ['Bánh Kẹo', 'Làm Sạch Cơ Thể', 'Mì, Bún, Topokki Ăn Liền', 'Snack, Ăn Vặt', 'Sốt, Gia Vị Các Loại', 'Sữa Chua, Váng Sữa', 'Sữa Nước'])]
cat_rules: (869, 9)


,ante_parent_cat,cons_parent_cat,co_count,ante_count,cons_count,support,confidence,lift,leverage
0,Điện Gia Dụng,Trà,8,24,311,0.001806,0.333333,4.748124,0.001426
1,Thịt Bò,"Xả Vải, Xịt Vải, Nước Tẩy",46,54,829,0.010384,0.851852,4.552115,0.008103
2,"Bột Giặt, Nước Giặt, Viên Giặt","Xả Vải, Xịt Vải, Nước Tẩy",652,812,829,0.147178,0.802956,4.290825,0.112878
3,"Xả Vải, Xịt Vải, Nước Tẩy","Bột Giặt, Nước Giặt, Viên Giặt",652,829,812,0.147178,0.786490,4.290825,0.112878
4,Thịt Bò,"Bánh Bao, Giò Chả, Đậu Hủ",13,54,252,0.002935,0.240741,4.232069,0.002241
5,Chăm Sóc Nam Giới,Chăm Sóc Nhà Cửa,21,23,989,0.004740,0.913043,4.089770,0.003581
6,Thú Cưng,Chăm Sóc Nhà Cửa,60,66,989,0.013544,0.909091,4.072065,0.010218
7,Thịt Bò,Chăm Sóc Nhà Cửa,49,54,989,0.011061,0.907407,4.064525,0.008340
8,Đồ Dùng Phòng Ngủ,"Bột Giặt, Nước Giặt, Viên Giặt",38,52,812,0.008578,0.730769,3.986832,0.006426
9,Thú Cưng,"Xả Vải, Xịt Vải, Nước Tẩy",49,66,829,0.011061,0.742424,3.967358,0.008273


Saved: Models\Rules\category_rules.csv


In [ ]:
item_rules = pd.read_csv(RULES_DIR / "item_rules.csv")
cat_rules  = pd.read_csv(RULES_DIR / "category_rules.csv")

# index rules for fast retrieval
item_by_ante = defaultdict(list)
for _, r in item_rules.iterrows():
    item_by_ante[str(r["antecedent_id"])].append(r)

cat_by_ante = defaultdict(list)
for _, r in cat_rules.iterrows():
    cat_by_ante[str(r["ante_parent_cat"])].append(r)

def recommend_also_item(main_df: pd.DataFrame, scores_vec: np.ndarray, k=5) -> pd.DataFrame:
    """
    Lấy top main items -> lấy consequents theo rule -> re-rank theo:
      score = 0.55*rule_score + 0.35*content_score(query) + 0.10*popularity
    """
    if main_df is None or main_df.empty:
        return pd.DataFrame(columns=["product_id_str","product_name_vi","category_name","brand_name","price","score"])

    seeds = main_df["product_id_str"].astype(str).tolist()[:5]
    cand_rows = []
    seen = set(seeds)

    for a in seeds:
        rules = item_by_ante.get(a, [])[:50]
        for r in rules:
            b = str(r["consequent_id"])
            if b in seen:
                continue
            seen.add(b)
            rule_score = float(r["lift"]) * float(r["confidence"])
            # content relevance from scores_vec (global over products)
            row_idx = prod_id_to_row.get(b, None)
            content_score = float(scores_vec[row_idx]) if row_idx is not None else 0.0
            pop = float(products.loc[row_idx, "popularity_norm"]) if row_idx is not None else 0.0
            score = 0.55*rule_score + 0.35*content_score + 0.10*pop
            cand_rows.append((b, score))

    if not cand_rows:
        return pd.DataFrame(columns=["product_id_str","product_name_vi","category_name","brand_name","price","score"])

    cand = pd.DataFrame(cand_rows, columns=["product_id_str","score"])
    cand = cand.merge(products[["product_id_str","product_name_vi","category_name","brand_name","price"]], on="product_id_str", how="left")
    cand = cand.sort_values("score", ascending=False).head(k).reset_index(drop=True)
    return cand

def recommend_also_category(q_info: dict, main_df: pd.DataFrame, scores_vec: np.ndarray, k=5) -> pd.DataFrame:
    """
    Dùng parent_category_name trong main -> lấy parent consequent -> lấy sản phẩm trong đó,
    rồi re-rank theo query relevance để KHÔNG lạc đề.
    """
    if main_df is None or main_df.empty:
        return pd.DataFrame(columns=["product_id_str","product_name_vi","category_name","brand_name","price","score"])

    seed_parents = main_df["parent_category_name"].fillna("").astype(str).tolist()[:5]
    seed_parents = [p for p in seed_parents if p]
    if not seed_parents:
        return pd.DataFrame(columns=["product_id_str","product_name_vi","category_name","brand_name","price","score"])

    # lấy top consequent parent categories
    parent_cands = []
    for p in seed_parents:
        for r in cat_by_ante.get(p, [])[:40]:
            cons = str(r["cons_parent_cat"])
            rule_score = float(r["lift"]) * float(r["confidence"])
            parent_cands.append((cons, rule_score))

    if not parent_cands:
        return pd.DataFrame(columns=["product_id_str","product_name_vi","category_name","brand_name","price","score"])

    parent_df = pd.DataFrame(parent_cands, columns=["parent_category_name","rule_score"])
    parent_df = parent_df.groupby("parent_category_name")["rule_score"].max().reset_index()
    parent_df = parent_df.sort_values("rule_score", ascending=False).head(6)

    # candidate products thuộc các parent này
    cand_prod = products[products["parent_category_name"].isin(parent_df["parent_category_name"])].copy()
    if cand_prod.empty:
        return pd.DataFrame(columns=["product_id_str","product_name_vi","category_name","brand_name","price","score"])

    # gán content score theo query
    cand_prod["row_idx"] = cand_prod["product_id_str"].astype(str).map(prod_id_to_row)
    cand_prod["content_score"] = cand_prod["row_idx"].apply(lambda i: float(scores_vec[i]) if pd.notna(i) else 0.0)
    cand_prod["pop"] = cand_prod["popularity_norm"].fillna(0).astype(float)

    # map rule_score theo parent_category
    rs_map = dict(zip(parent_df["parent_category_name"], parent_df["rule_score"]))
    cand_prod["rule_score"] = cand_prod["parent_category_name"].map(rs_map).fillna(0.0)

    # budget constraint
    bmax = q_info.get("constraints", {}).get("budget_max", None)
    if isinstance(bmax, (int,float)) and bmax and bmax > 0:
        cand_prod = cand_prod[cand_prod["price"].fillna(0).astype(int) <= int(bmax)].copy()

    # exclude
    mask = apply_excludes_mask(cand_prod, q_info.get("exclude_terms", []))
    cand_prod = cand_prod[mask].copy()

    # final score
    cand_prod["score"] = 0.55*cand_prod["rule_score"] + 0.35*cand_prod["content_score"] + 0.10*cand_prod["pop"]
    out = cand_prod.sort_values("score", ascending=False).head(k)
    return out[["product_id_str","product_name_vi","category_name","brand_name","price","score"]].reset_index(drop=True)

# Demo compare also-like (item vs category)
q_info = qi.analyze("cần mua đồ giặt đồ và nước xả vải dưới 200k")
main_df, scores_vec = recommend_content(q_info, k=8, return_scores=True)
also_item = recommend_also_item(main_df, scores_vec, k=5)
also_cat  = recommend_also_category(q_info, main_df, scores_vec, k=5)

print("MAIN:")
display(main_df)
print("ALSO ITEM:")
display(also_item)
print("ALSO CATEGORY:")
display(also_cat)

MAIN:


,product_id_str,product_name_vi,category_name,parent_category_name,brand_name,price,score
6218,68312cdb419bc51ab2ee7646,Bột Giặt Surf Hương Nước Hoa Duyên Dáng Túi 5.3kg,Bột Giặt,"Bột Giặt, Nước Giặt, Viên Giặt",Surf,149900,0.552703
4112,68312752419bc51ab2ee6d4b,Nước Giặt Comfort Dưỡng Vải Thanh Lịch Túi 3.8kg,Nước Giặt,"Bột Giặt, Nước Giặt, Viên Giặt",Comfort,143900,0.525998
6221,68312cde419bc51ab2ee7649,Bột Giặt Surf Hương Nước Hoa Quyến Rũ Túi 5.3kg,Bột Giặt,"Bột Giặt, Nước Giặt, Viên Giặt",Surf,149900,0.522944
4145,68312769419bc51ab2ee6d71,Nước Giặt Lix Đậm Đặc Hương Nước Hoa Túi 3.2kg,Nước Giặt,"Bột Giặt, Nước Giặt, Viên Giặt",Lix,95900,0.521625
4172,6831277a419bc51ab2ee6d8f,Nước Giặt Surf Trắng Sạch Hương Hoa Cỏ Diệu Kỳ...,Nước Giặt,"Bột Giặt, Nước Giặt, Viên Giặt",Unilever,94900,0.518418
4140,68312764419bc51ab2ee6d6b,Nước Giặt Blue Kháng Khuẩn Hương Hoa Thạch Thả...,Nước Giặt,"Bột Giặt, Nước Giặt, Viên Giặt",Blue,133900,0.516933
4183,6831277f419bc51ab2ee6d9d,Nước Giặt Surf Hương Sương Mai Dịu Mát Túi 3.3kg,Nước Giặt,"Bột Giặt, Nước Giặt, Viên Giặt",Surf,94900,0.511568
4135,68312761419bc51ab2ee6d65,Nước Giặt Blue Kháng Khuẩn Muối Hồng Himalaya ...,Nước Giặt,"Bột Giặt, Nước Giặt, Viên Giặt",Blue,133900,0.510995


ALSO ITEM:


,product_id_str,score,product_name_vi,category_name,brand_name,price
0,68312802419bc51ab2ee6e77,17.752481,Nước Xả Vải Lix Sạch Thơm Ngàn Hoa Túi 2.2L,"Xả Vải, Xịt Vải",Lix,94900


ALSO CATEGORY:


,product_id_str,product_name_vi,category_name,brand_name,price,score
0,683127f2419bc51ab2ee6e57,Nước Xả Vải Blue Đậm Đặc Hương Hoa Hương Thảo ...,"Xả Vải, Xịt Vải",Blue,135900,2.050697
1,683127e0419bc51ab2ee6e39,Nước Xả Vải Downy Hương Hoa Oải Nước Pháp 3L,"Xả Vải, Xịt Vải",Downy,192900,2.050547
2,683127f2419bc51ab2ee6e59,Nước Xả Vải Blue Đậm Đặc Hương Thanh Xuân Túi ...,"Xả Vải, Xịt Vải",Blue,135900,2.043229
3,683127e0419bc51ab2ee6e3a,Nước Xả Vải Downy Hương Đam Mê Túi 2.5L,"Xả Vải, Xịt Vải",P&G,188900,2.041555
4,683127ed419bc51ab2ee6e4d,Nước Xả Vải Good Care Hương Lavender Chai 3L,"Xả Vải, Xịt Vải",Good Care,152000,2.037765


In [12]:
class TraditionalRecommender:
    """
    Pipeline:
      1) QueryInterpreterLite -> q_info (intent, constraints, excludes, include_terms)
      2) Content-based -> MAIN (theo ý user)
      3) Also-like ITEM (mua chung) re-rank theo query
      4) Also-like CATEGORY (mua chung theo ngành hàng) re-rank theo query
    """
    def __init__(self, interpreter: QueryInterpreterLite):
        self.interpreter = interpreter

    def recommend(self, query_raw: str, k_main=10, k_item=5, k_cat=5):
        q_info = self.interpreter.analyze(query_raw)
        main_df, scores_vec = recommend_content(q_info, k=k_main, return_scores=True)
        also_item_df = recommend_also_item(main_df, scores_vec, k=k_item)
        also_cat_df  = recommend_also_category(q_info, main_df, scores_vec, k=k_cat)
        return main_df, also_item_df, also_cat_df, q_info

rec_sys = TraditionalRecommender(qi)

def demo(query: str):
    main_df, also_item_df, also_cat_df, q_info = rec_sys.recommend(query, k_main=8, k_item=5, k_cat=5)
    print("="*90)
    print("YÊU CẦU NGƯỜI DÙNG:", query)
    print("\n>>> PHÂN TÍCH:")
    for k in ["query_norm","intent","matched_intents","exclude_terms","constraints","include_terms"]:
        print(f"  - {k:14s}:", q_info.get(k))
    print("\n>>> GỢI Ý CHÍNH (Content-based):")
    display(main_df)
    print("\n>>> ALSO-LIKE (Item rules + relevance):")
    display(also_item_df)
    print("\n>>> ALSO-LIKE (Category rules + relevance):")
    display(also_cat_df)

demo("muốn nấu canh chua cho 4 người ăn, đừng mua cá lóc")
demo("cần mua đồ giặt đồ và nước xả vải dưới 200k")
demo("tối nay ăn vặt nhẹ xem phim")
demo("tìm sữa rửa mặt cho da dầu mụn")


YÊU CẦU NGƯỜI DÙNG: muốn nấu canh chua cho 4 người ăn, đừng mua cá lóc

>>> PHÂN TÍCH:
  - query_norm    : muốn nấu canh chua cho 4 người ăn, đừng mua cá lóc
  - intent        : cook
  - matched_intents: ['cook']
  - exclude_terms : ['cá lóc']
  - constraints   : {'budget_max': None, 'quantity_people': 4}
  - include_terms : ['nguyên liệu', 'gia vị', 'rau', 'thịt', 'cá', 'nước mắm', 'nấu', 'canh', 'chua', '4', 'người', 'ăn', 'lóc']

>>> GỢI Ý CHÍNH (Content-based):


,product_id_str,product_name_vi,category_name,parent_category_name,brand_name,price,score
3510,683125ca419bc51ab2ee6ab1,Xốt Gia Vị Hoàn Chỉnh Barona Nấu Canh Chua Gói...,Gia Vị Hoàn Chỉnh,"Sốt, Gia Vị Các Loại",Barona,12600,0.338386
3532,683125da419bc51ab2ee6ac9,Gia Vị Hoàn Chỉnh Knorr Canh Chua 30g,Gia Vị Hoàn Chỉnh,"Sốt, Gia Vị Các Loại",Knorr,7400,0.325106
8046,683131c4419bc51ab2ee7e1e,Cá Ba Sa Nấu Canh Chua 365 Fresh (ea),Thực Phẩm Sơ Chế,NaN,365 Fresh,41000,0.313363
4998,683129ab419bc51ab2ee712a,Lá Giang 100G (ea),Rau Lá,Rau Củ,UNKNOWN_BRAND,13500,0.297201
3743,6831265f419bc51ab2ee6baf,Xốt Gia Vị Nấu Canh Tương Đậu Rau Dadam 140G,Gia Vị Hoàn Chỉnh,"Sốt, Gia Vị Các Loại",Dadam,45900,0.282474
3081,683124ab419bc51ab2ee68bd,Mắm Cá Sặc Sông Hương Foods Hũ 400g,Mắm Các Loại,"Mắm, Rau Củ Muối",Sông Hương,83100,0.259405
8043,683131c3419bc51ab2ee7e1b,Canh Chua Cá Diêu Hồng,Thực Phẩm Sơ Chế,NaN,365 Fresh,59000,0.257313
1044,68311f67419bc51ab2ee5fa3,Nguyên Liệu Canh Ngót An Lạc 450G,Rau Củ Khác,Rau Củ,An Lạc,29900,0.256606



>>> ALSO-LIKE (Item rules + relevance):


,product_id_str,product_name_vi,category_name,brand_name,price,score



>>> ALSO-LIKE (Category rules + relevance):


...

,product_id_str,product_name_vi,category_name,parent_category_name,brand_name,price,score
3245,68312513419bc51ab2ee6981,Sữa Rửa Mặt Hazeline Chiết Xuất Matcha Tràm Tr...,Sữa Rửa Mặt,Làm Sạch,Unilever,28000,0.460001
3244,68312513419bc51ab2ee6980,Sữa Rửa Mặt Hatomugi Ý Dĩ Ngừa Mụn Làm Sáng Da...,Sữa Rửa Mặt,Làm Sạch,Hatomugi,93000,0.456723
3233,6831250c419bc51ab2ee6972,Sữa Rửa Mặt Simple Giúp Da Sạch Thoáng Trông K...,Sữa Rửa Mặt,Làm Sạch,Simple,140000,0.443729
3257,68312519419bc51ab2ee698f,Sữa Rửa Mặt Neutrogena Làm Sạch Sâu Dạng Gel C...,Sữa Rửa Mặt,Làm Sạch,Neutrogena,140000,0.442842
8432,683132cc419bc51ab2ee7fc2,Sữa Rửa Mặt Pond's Sáng Da Đều Màu 100g,Dưỡng Da,Chăm Sóc Da Mặt,Unilever,89900,0.441689
3213,68312500419bc51ab2ee695b,Gel Rửa Mặt Simple Thanh Khiết Giảm Bóng Nhờn ...,Sữa Rửa Mặt,Làm Sạch,Simple,140000,0.439939
8385,683132af419bc51ab2ee7f90,Kem Rửa Mặt & Mặt Nạ Acnes Ngăn Ngừa Mụn Đầu Đ...,Dưỡng Da,Chăm Sóc Da Mặt,Acnes,84000,0.435051
3256,68312518419bc51ab2ee698d,Sữa Rửa Mặt Bioré Thanh Lọc Da - Dưỡng Ẩm & Mị...,Sữa Rửa Mặt,Làm Sạch,Bioré,55000,0.432262



>>> ALSO-LIKE (Item rules + relevance):


,product_id_str,product_name_vi,category_name,brand_name,price,score



>>> ALSO-LIKE (Category rules + relevance):


,product_id_str,product_name_vi,category_name,brand_name,price,score
0,68312eee419bc51ab2ee7990,Dầu Xả Dưỡng Tóc Mise En Scène Perfect Serum S...,Dầu Xả,Mise En Scène,230000,0.656575
1,68312f09419bc51ab2ee79c7,Dầu Xả Dưỡng Tóc Mise En Scène Perfect Serum O...,Dầu Xả,Mise En Scène,230000,0.656257
2,68312eed419bc51ab2ee798d,Dầu Xả Cocoon Chiết Xuất Bưởi Cung Cấp Dưỡng C...,Dầu Xả,Cocoon,195000,0.652817
3,68312f0d419bc51ab2ee79ce,Serum Cho Tóc Hư Tổn TRESemmé Bond Repair Giảm...,Dầu Xả,Unilever,187900,0.642615
4,68312f01419bc51ab2ee79b8,Dầu Xả TRESemmé Keratin Cho Tóc Khô Vào Nếp Su...,Dầu Xả,Unilever,231000,0.639653


In [ ]:
import random

def ndcg_at_k(recs: List[str], gt: set, k: int) -> float:
    dcg = 0.0
    for i, pid in enumerate(recs[:k], start=1):
        rel = 1.0 if pid in gt else 0.0
        dcg += rel / math.log2(i + 1)
    # ideal: tất cả gt lên đầu
    ideal_hits = min(len(gt), k)
    idcg = sum(1.0 / math.log2(i + 1) for i in range(1, ideal_hits + 1))
    return dcg / idcg if idcg > 0 else 0.0

def map_at_k(recs: List[str], gt: set, k: int) -> float:
    hits = 0
    s = 0.0
    for i, pid in enumerate(recs[:k], start=1):
        if pid in gt:
            hits += 1
            s += hits / i
    return s / min(len(gt), k) if gt else 0.0

def eval_leave_one_out(sample_n=300, k=10, seed=42):
    """
    Offline eval theo chuẩn retail khi không có query thật:
    - Tạo pseudo-query từ 1 sản phẩm trong bill: "mua <tên_sp>"
    - Ground-truth = các sản phẩm còn lại trong bill
    Eval 3 thứ:
      A) MAIN(Content)         : khả năng tìm đúng sản phẩm liên quan từ query
      B) ALSO_ITEM(item_rules) : khả năng mua-chung theo sản phẩm (re-ranked theo query)
      C) ALSO_CAT(cat_rules)   : khả năng mua-chung theo ngành hàng (re-ranked theo query)
    """
    rng = random.Random(seed)
    # bill hợp lệ: >=2 items
    bill_to_items = tx_long.groupby("bill_id")["product_id_str"].apply(lambda s: sorted(set(s.astype(str)))).to_dict()
    valid = [(bid, it) for bid, it in bill_to_items.items() if len(it) >= 2]
    rng.shuffle(valid)
    valid = valid[:sample_n]

    hr_main, recall_main, ndcg_main, map_main = [], [], [], []
    hr_item, recall_item, ndcg_item, map_item = [], [], [], []
    hr_cat,  recall_cat,  ndcg_cat,  map_cat  = [], [], [], []

    name_map = products.set_index("product_id_str")["product_name_vi"].astype(str).to_dict()

    for bid, items in valid:
        anchor = rng.choice(items)
        gt = set([x for x in items if x != anchor])
        q = "mua " + name_map.get(anchor, "")

        main_df, also_item_df, also_cat_df, _ = rec_sys.recommend(q, k_main=k, k_item=k, k_cat=k)

        rec_main = main_df["product_id_str"].astype(str).tolist()
        rec_item = also_item_df["product_id_str"].astype(str).tolist()
        rec_cat  = also_cat_df["product_id_str"].astype(str).tolist()

        def pack(recs):
            hit = 1.0 if any(x in gt for x in recs[:k]) else 0.0
            recall = len(set(recs[:k]) & gt) / len(gt) if gt else 0.0
            return hit, recall, ndcg_at_k(recs, gt, k), map_at_k(recs, gt, k)

        a = pack(rec_main); b = pack(rec_item); c = pack(rec_cat)
        hr_main.append(a[0]); recall_main.append(a[1]); ndcg_main.append(a[2]); map_main.append(a[3])
        hr_item.append(b[0]); recall_item.append(b[1]); ndcg_item.append(b[2]); map_item.append(b[3])
        hr_cat.append(c[0]);  recall_cat.append(c[1]);  ndcg_cat.append(c[2]);  map_cat.append(c[3])

    return {
        "n_eval": len(valid),
        f"HitRate@{k}_MAIN(content)": float(np.mean(hr_main)),
        f"Recall@{k}_MAIN(content)": float(np.mean(recall_main)),
        f"NDCG@{k}_MAIN(content)": float(np.mean(ndcg_main)),
        f"MAP@{k}_MAIN(content)": float(np.mean(map_main)),
        f"HitRate@{k}_ALSO_ITEM": float(np.mean(hr_item)),
        f"Recall@{k}_ALSO_ITEM": float(np.mean(recall_item)),
        f"NDCG@{k}_ALSO_ITEM": float(np.mean(ndcg_item)),
        f"MAP@{k}_ALSO_ITEM": float(np.mean(map_item)),
        f"HitRate@{k}_ALSO_CAT": float(np.mean(hr_cat)),
        f"Recall@{k}_ALSO_CAT": float(np.mean(recall_cat)),
        f"NDCG@{k}_ALSO_CAT": float(np.mean(ndcg_cat)),
        f"MAP@{k}_ALSO_CAT": float(np.mean(map_cat)),
    }

print(eval_leave_one_out(sample_n=300, k=10))

{'n_eval': 300, 'HitRate@10_MAIN(content)': 0.07, 'Recall@10_MAIN(content)': 0.009074148074148074, 'NDCG@10_MAIN(content)': 0.008517901906524733, 'MAP@10_MAIN(content)': 0.0027457671957671955, 'HitRate@10_ALSO_ITEM': 0.03, 'Recall@10_ALSO_ITEM': 0.004822751322751323, 'NDCG@10_ALSO_ITEM': 0.005440851054630933, 'MAP@10_ALSO_ITEM': 0.0022137566137566136, 'HitRate@10_ALSO_CAT': 0.13333333333333333, 'Recall@10_ALSO_CAT': 0.016553307803307802, 'NDCG@10_ALSO_CAT': 0.014775726596722747, 'MAP@10_ALSO_CAT': 0.004463897287309986}
